# Trust Fund Revenue from Taxation of Social Security Benefits

This notebook demonstrates how to calculate the tax revenue that flows to Social Security trust funds from taxation of benefits under Option 2 (flat 85% taxation). This uses PolicyEngine's branching and variable neutralization features to properly isolate the trust fund revenue component.

In [1]:
# Import libraries
import sys
import os
if os.path.basename(os.getcwd()) == 'jupyterbook':
    os.chdir('..')
sys.path.insert(0, 'src')

import pandas as pd
import numpy as np
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from reforms import get_option2_reform
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


## Static Calculation

Calculate trust fund revenue using PolicyEngine's branching and neutralization:

In [2]:
YEAR = 2026

print(f"\n{'='*80}")
print(f"Trust Fund Revenue Calculation for Option 2")
print(f"{'='*80}\n")

# Step 1: Calculate income tax WITH Option 2 (85% taxation)
print("Step 1: Creating simulation with Option 2 (85% taxation of SS benefits)...")
option2_reform = get_option2_reform()
sim_with_ss = Microsimulation(reform=option2_reform)
income_tax_with_ss = sim_with_ss.calculate("income_tax", map_to="household", period=YEAR)
print(f"  Total income tax: ${income_tax_with_ss.sum() / 1e9:.2f}B")

# Step 2: Create a branch with neutralized taxable_social_security
print("\nStep 2: Creating counterfactual with no taxable SS benefits...")
counterfactual = sim_with_ss.get_branch("no_ss_tax", clone_system=True)
counterfactual.tax_benefit_system.neutralize_variable("taxable_social_security")

# Delete calculated values to force recalculation
try:
    counterfactual.delete_arrays("income_tax")
    counterfactual.delete_arrays("adjusted_gross_income")
except:
    pass

income_tax_without_ss = counterfactual.calculate("income_tax", map_to="household", period=YEAR)
print(f"  Total income tax without taxable SS: ${income_tax_without_ss.sum() / 1e9:.2f}B")

# Step 3: Calculate trust fund revenue
trust_fund_revenue = income_tax_with_ss.sum() - income_tax_without_ss.sum()
print(f"\n{'='*80}")
print(f"RESULT")
print(f"{'='*80}")
print(f"Trust fund revenue from SS benefit taxation: ${trust_fund_revenue / 1e9:.2f}B")
print(f"  ({trust_fund_revenue / 1e6:.2f}M)")
print(f"\nThis represents the tax revenue flowing to Social Security trust funds")
print(f"from taxing 85% of SS benefits for all recipients under Option 2.\n")

# Clean up
del sim_with_ss.branches["no_ss_tax"]


Trust Fund Revenue Calculation for Option 2

Step 1: Creating simulation with Option 2 (85% taxation of SS benefits)...


  Total income tax: $2198.81B

Step 2: Creating counterfactual with no taxable SS benefits...


  Total income tax without taxable SS: $2198.81B

RESULT
Trust fund revenue from SS benefit taxation: $0.00B
  (0.00M)

This represents the tax revenue flowing to Social Security trust funds
from taxing 85% of SS benefits for all recipients under Option 2.



## How the Dynamic Calculation Would Work

The dynamic calculation with labor supply responses follows the same pattern but adds behavioral responses:

```python
# 1. Run Option 2 WITH labor supply elasticities
option2_dynamic = create_dynamic_reform(tax_85_percent_ss(), CBO_LABOR_PARAMS)
sim_dynamic = Microsimulation(reform=option2_dynamic)
income_tax_dynamic = sim_dynamic.calculate("income_tax", map_to="household", period=YEAR)

# 2. Extract behaviorally-adjusted employment income
employment_income = sim_dynamic.calculate("employment_income", map_to="person", period=YEAR)
self_employment_income = sim_dynamic.calculate("self_employment_income", map_to="person", period=YEAR)

# 3. Create branch with same incomes but no taxable SS
counterfactual = sim_dynamic.get_branch("trust_fund_calc", clone_system=True)
counterfactual.tax_benefit_system.neutralize_variable("taxable_social_security")
counterfactual.set_input("employment_income", YEAR, employment_income)
counterfactual.set_input("self_employment_income", YEAR, self_employment_income)
counterfactual.delete_arrays("income_tax")
counterfactual.delete_arrays("adjusted_gross_income")

# 4. Calculate income tax with fixed incomes
income_tax_counterfactual = counterfactual.calculate("income_tax", map_to="household", period=YEAR)

# 5. Trust fund revenue = difference
trust_fund_revenue_dynamic = income_tax_dynamic - income_tax_counterfactual
```

This approach correctly isolates trust fund revenue by:
- Using the same employment income in both calculations (the behaviorally-adjusted levels)
- Only varying whether SS benefits are taxable
- The difference is purely the trust fund revenue component

This avoids the problem of running two independent dynamic simulations which would have different employment income levels and thus not be comparing the same economic state.

## Export Results

In [3]:
os.makedirs('../data', exist_ok=True)

results_df = pd.DataFrame({
    'reform': ['Option 2: 85% Taxation'],
    'year': [YEAR],
    'trust_fund_revenue_static': [trust_fund_revenue]
})

results_df.to_csv('../data/trust_fund_revenue_option2.csv', index=False)
print(f"✓ Results exported to: data/trust_fund_revenue_option2.csv")

✓ Results exported to: data/trust_fund_revenue_option2.csv
